# HW-3
### Q1 GDA

In [1]:
import numpy as np
from scipy.stats import false_discovery_control
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
data = np.loadtxt("spambase/spambase.data", delimiter=",")
X = data[:, :-1]
y = data[:, -1].astype(int)

scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
class GDA:
    def __init__(self, shared_covariance = False):
        self.shared_covariance = shared_covariance
        self.priors = {}
        self.means = {}
        self.covariances = {}
        self.shared_cov = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        n_samples = X.shape[0]
        n_features = X.shape[1]

        for c in self.classes:
            X_c = X[y==c]

            # prior:
            self.priors[c] = len(X_c) / n_samples

            # mean vector:
            self.means[c] = np.mean(X_c, axis=0)

            # cov for QDA
            if not self.shared_covariance:
                self.covariances[c] = np.cov(X_c.T, bias=False)

                # reg param
                self.covariances[c] += 1e-6 * np.eye(n_features)

        # single shared cov for LDA:
        if self.shared_covariance:
            self.shared_cov = np.cov(X.T, bias=False)

            # reg param:
            self.shared_cov += 1e-6 * np.eye(n_features)

    def predict(self, X):
        """predicting class for each instance using log prob"""

        predictions = []

        for x in X:
            log_probs = {}

            for c in self.classes:
                if self.shared_covariance:
                    cov = self.shared_cov
                else:
                    cov = self.covariances[c]

                mean_c = self.means[c]
                diff = x - mean_c



